In [62]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [63]:
env = GridworldEnv()

In [72]:
def policy_eval(pi, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    gamma = discount_factor
    i = 0 # Number of iterations till convergance 
    while True:
        # TODO: Implement!  
        Vold = list(V) # CANT assign Vold = V because it would assign the same memory position to both  
        
        for s in range(env.nS):            
            Vaux = 0 # used to compute Expectation over all actions  
            
            for a in range(env.nA): # == env.action_space.n
                P,St1,Ra =  env.P[s][a][0][0:3] # No need to iterate for each s'<-- deterministic MDP
                Vaux += pi[s,a]*( Ra + gamma*(P*V[St1]) ) ## In-Place-DP
#                 Vaux += pi[s,a]*( Ra + gamma*(P*Vold[St1]) ) ## Normal DP (two copies of value function)
            V[s] = Vaux # update new value for state - s
            
        i+=1
        if all([compare<theta for compare in abs(V-Vold)]):
            break
            
    return np.array(V),i

In [89]:
random_policy = np.ones([env.nS, env.nA]) / env.nA # random uniform policy
v,i = policy_eval(random_policy, env) # call policy_eval methods
print(i)
v

141


array([  0.        , -13.99993529, -19.99990698, -21.99989761,
       -13.99993529, -17.9999206 , -19.99991379, -19.99991477,
       -19.99990698, -19.99991379, -17.99992725, -13.99994569,
       -21.99989761, -19.99991477, -13.99994569,   0.        ])

In [77]:
#### Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])# Correct Vpi
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)